# PyTorch Metric Learning
See the documentation [here](https://kevinmusgrave.github.io/pytorch-metric-learning/)

## Install the packages

In [1]:
!pip install pytorch-metric-learning
!pip install -q faiss-gpu
!pip install scikit-image==0.19.0

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
!cp gdrive/My\ Drive//mt/src/helpers.py .
!cp gdrive/My\ Drive//mt/src/models.py .
!cp gdrive/My\ Drive//mt/src/features.py .
!cp gdrive/My\ Drive//mt/src/0_13241_4467NR.png .

In [21]:
from pathlib import Path

train_data = Path("./train/")

if not train_data.is_dir():
    !unzip gdrive/My\ Drive//mt/data/train.zip
    !unzip gdrive/My\ Drive//mt/data/test.zip
    !unzip gdrive/My\ Drive//mt/data/val.zip

## Import the packages

In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
import toml
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.utils import common_functions as c_f
from pytorch_metric_learning.utils.inference import InferenceModel, MatchFinder
import helpers
import models
import features
import PIL

## Create helper functions

In [23]:
def print_decision(is_match):
    if is_match:
        print("Same class")
    else:
        print("Different class")


mean = [0.6143, 0.6884, 0.7665]
std = [0.2909, 0.2548, 0.2122]

inv_normalize = transforms.Normalize(
    mean=[-m / s for m, s in zip(mean, std)], std=[1 / s for s in std]
)


def imshow(img, figsize=(8, 4)):
    img = inv_normalize(img)
    npimg = img.numpy()
    plt.figure(figsize=figsize)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [33]:
config = toml.load('./gdrive/MyDrive/mt/conf/conf.toml')
setting = config.get('settings')
param = config.get('params')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)

In [39]:
dataset = features.PypyrusDataset(data=setting['path_test'],
                               csv=setting['csv'],
                               mode='test',
                               transform=transform,
                               debug=False,
                               batch_size=64,)

In [40]:
labels_to_indices = c_f.get_labels_to_indices(dataset.targets)

In [41]:
len(labels_to_indices)

100

## Create the dataset and load the trained model

In [42]:
output_path = './gdrive/MyDrive/mt/out/' + 'Dataset_3_Scheduler_Densenet121_MultiSimilarityLoss_MultiSimilarityMiner/saved_models/'

trunk_model = output_path + 'trunk_best13.pth'
embedder_model = output_path + 'embedder_best13.pth'
classifier_model = output_path + 'classifier_best13.pth'

In [43]:
trunk = torchvision.models.densenet121(pretrained=False)
trunk_output_size = trunk.classifier.in_features
trunk.classifier = c_f.Identity()
trunk.to(torch.device('cuda'))
trunk.load_state_dict(torch.load(trunk_model))

<All keys matched successfully>

In [44]:
embedder = models.MLP([trunk_output_size, 64]).to(torch.device('cuda'))
embedder.load_state_dict(torch.load(embedder_model))

<All keys matched successfully>

## Create the InferenceModel wrapper

In [45]:
match_finder = MatchFinder(distance=CosineSimilarity(), threshold=0.9)

In [46]:
inference_model = InferenceModel(trunk,
                embedder=embedder,
                match_finder=match_finder,
                normalize_embeddings=True,)

## Get nearest neighbors of a query

In [47]:
# create faiss index
inference_model.train_knn(dataset)

In [48]:
sacc = []
for i, (key, papyid) in enumerate(labels_to_indices.items()):
    top_1 = 0
    for j in range(len(papyid)):
        
        img = dataset[papyid[j]][0].unsqueeze(0)
        #print(type(img))
        #print(img.shape)
        #print(f"query image {keys[i]}")
        #imshow(torchvision.utils.make_grid(img))
        distances, indices = inference_model.get_nearest_neighbors(img, k=10)
        nearest_imgs = [dataset[i][0] for i in indices.cpu()[0]]
        labels = [dataset[i][1] for i in indices.cpu()[0]]
        if labels[0] == key:
            top_1 = top_1 + 1
        #print("nearest images")
        #imshow(torchvision.utils.make_grid(nearest_imgs))
        #print(labels)
    class_top_1 = top_1 / (j +1)
    acc.append(class_top_1)
    print(f"Top-1 Acc of Papy {key}: {top_1} / {j+1} = {class_top_1}")
print(f"Overall Top-1 Acc: {np.array(acc).mean()}")
        
#get_top_1()

Top-1 Acc of Papy 17859: 80 / 80 = 1.0
Top-1 Acc of Papy 9313: 40 / 40 = 1.0
Top-1 Acc of Papy 13280: 40 / 40 = 1.0
Top-1 Acc of Papy 4444: 120 / 120 = 1.0
Top-1 Acc of Papy 12226: 40 / 40 = 1.0
Top-1 Acc of Papy 15170: 80 / 80 = 1.0
Top-1 Acc of Papy 14640: 58 / 58 = 1.0
Top-1 Acc of Papy 15632: 40 / 40 = 1.0
Top-1 Acc of Papy 17860: 80 / 80 = 1.0
Top-1 Acc of Papy 12617: 120 / 120 = 1.0
Top-1 Acc of Papy 13884: 160 / 160 = 1.0
Top-1 Acc of Papy 5164: 60 / 60 = 1.0
Top-1 Acc of Papy 13897: 180 / 180 = 1.0
Top-1 Acc of Papy 12840: 51 / 100 = 0.51
Top-1 Acc of Papy 13971: 70 / 70 = 1.0
Top-1 Acc of Papy 8601: 48 / 48 = 1.0
Top-1 Acc of Papy 17077: 80 / 80 = 1.0
Top-1 Acc of Papy 14817: 80 / 80 = 1.0
Top-1 Acc of Papy 13682: 72 / 72 = 1.0
Top-1 Acc of Papy 12810: 80 / 80 = 1.0
Top-1 Acc of Papy 16613: 60 / 60 = 1.0
Top-1 Acc of Papy 9312: 60 / 60 = 1.0
Top-1 Acc of Papy 11346: 40 / 40 = 1.0
Top-1 Acc of Papy 9872: 40 / 40 = 1.0
Top-1 Acc of Papy 17060: 40 / 40 = 1.0
Top-1 Acc of Papy 115